# Lab 4 - Introduction to Match-action Tables (Part 1)
<div style="text-align: justify;">
This lab will walk you through creating an experiment that contains a P4-DPDK programmable pipeline. This lab describes match-action tables and how to define them in a P4 program. It then explains the different types of matching that can be performed on keys. The lab further shows how to track the misses/hits of a table key while receiving a packet.
<figure style="text-align: center;">
<img src="./labs_files/lab4/figs/0_00_fabric_topology.png" width="550px"><br>

# Introduction to control blocks

<div style="text-align: justify;">
P4-DPDK supports the PNA architecture which consists of the parser, the pre-control, the main control, and the deparser. The Main Control block is essential for processing a packet. It is where the code would be written for the packet processing logic. It aims to transform headers, update stateful elements like counters, meters, and registers, and optionally associate additional user-defined metadata with the packet [<a href="#References">1,2</a>]. For example, a control block for layer-3 forwarding may require a forwarding table that is indexed by the destination IP address. The control block may include actions to forward a packet when a hit occurs, and to drop the packet otherwise. Figure 1 shows the basic structure of a control block. </div>   <br> 

<figure style="text-align: center;">
  <img src="./labs_files/lab4/figs/0_intro_1_control.png" width="450" style="display: block; margin: 0 auto;">
  <figcaption>Figure 1. Control blocks.</figcaption>
</figure>

## Tables

Tables are essential components that define the processing behavior of a packet inside the switch. A table is specified in the P4 program and has one or more entries (rows) that are populated by the control plane. An entry contains a key, an action, and action data.  

•	Key: it is used for lookup operations. A key is built for the incoming packet using one or more header fields (e.g., destination IP address) or metadata (e.g., ingress port ID, egress port ID) and then looks up that value in the table.  
•	Action: once a match occurs, the action specified in the entry is performed by the arithmetic logic unit. Actions are simple operations such as modifying a header field, forwarding the packet to an egress port, and dropping the packet. The P4 program contains the possible actions.  
•	Action data: it can be considered as parameter/s used along with the action. For example, the action data may represent the port number the switch must use to forward the packet. The action data is populated by the control plane.  

## Match types

<div style="text-align: justify;">
There are three types of matching: exact match, Longest Prefix match (LPM), and ternary match. They are defined in the standard library (core.p4 [<a href="#References">3</a>]). Note that architectures may define and implement additional match types. For example, the PNA [<a href="#References">1,4</a>] also has matching based on ranges and selectors. In this lab, we will discuss exact matching. </div>   <br> 

## Exact match

<div style="text-align: justify;">
Assume that the exact match lookup is used to search for a specific value of an entry in a table. Assume that Table 1 matches on the destination IP address. If an incoming packet has 10.0.0.2 as the destination IP address, then it will match against the second entry and the P4 program will forward the packet using port 1 as the egress port of the pipeline. </div>   <br>

<table style="width: 50%; font-size: 14px;">
  <caption>Table 1. Exact match table.</caption>
  <thead>
    <tr style="background-color: #f0f0f0;">
      <th>Key</th>
      <th>Action</th>
      <th>Action data</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="padding: 10px; text-align: center;">10.0.0.1</td>
      <td style="padding: 10px; text-align: center;">forward</td>
      <td style="padding: 10px; text-align: center;">port 0</td>
    </tr>
    <tr>
      <td style="padding: 10px; text-align: center;">10.0.0.2</td>
      <td style="padding: 10px; text-align: center;">forward</td>
      <td style="padding: 10px; text-align: center;">port 1</td>
    </tr>
    <tr>
      <td style="padding: 10px; text-align: center;">default</td>
      <td style="padding: 10px; text-align: center;">drop</td>
      <td style="padding: 10px; text-align: center;"> </td>
    </tr>
  </tbody>
</table>

Figure 2 shows the main control block portion of a P4 program. Two actions are defined, ```drop``` and ```forward```. The ```drop``` action (lines 6 - 8) invokes the ```drop_packet``` function, causing the packet to be dropped. The ```forward``` action (lines 9 - 11) accepts as input (i.e., action data) the destination port. This parameter is inserted by the control plane and updated in the packet during the ingress processing. In line 10, the P4 program assigns the egress port defined by the control plane as an input to the ```send_to_port``` extern function. It is used to direct a packet to a specified network port. Lines 12-22 implement a table named ```forwarding```. The match is against the destination IP address using the exact lookup method. The actions associated with the table are forward and drop. The default action which is invoked when there is a miss is the drop action. The maximum number of entries a table can support is configured manually by the programmer (i.e., 1024 entries, see line 20). Note, however, that the number of entries is limited by the amount of memory in the switch. The control block starts executing from the apply statement (see lines 23-28) which contains the control logic. In this program, the ```forwarding``` table is enabled when the incoming packet has a valid IPv4 header. Otherwise, the packet is dropped.

<figure style="text-align: center;">
  <img src="./labs_files/lab4/figs/0_intro_2_p4_control.png" width="600" style="display: block; margin: 0 auto;">
  <figcaption>Figure 2. Main control block portion of a P4 program. The code implements a match-action table with exact match lookup.</figcaption>
</figure>

## Add_on_miss capability

<div style="text-align: justify;">
The add_on_miss [<a href="#References">1</a>]  table property is uniquely compatible with the P4 PNA. This feature helps by adding rules to a table whenever a match is not found without the control plane’s contribution. Note that this feature is only applicable with exact matching. The add_on_miss parameter takes a boolean value. If set to true, the default action executed adds an entry to the table when a match is not found. Therefore, the new table entry will be a successful match when the next packet is processed.   </div>   <br> 

# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

# Step 2: Importing the FABlib library

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Creating the experiment slice

The following creates a node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Creating a slice
The code below creates a new slice with the name "P4DPDK_lab4_vnic"

In [3]:
slice = fablib.new_slice(name="P4DPDK_lab4_vnic")

### Step 3.2: Defining the sites
The code below requests a random site from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [4]:
site1= fablib.get_random_sites(count=1, filter_function=lambda x: x['cores_available'] > 4 and x['ram_available'] > 8 and x['disk_available'] > 20)[0]

print (f'The selected site is {site1}')

The selected site is NEWY


### Step 3.3: Creating the nodes
The code below creates one node (server1) which uses the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1

In [5]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [6]:
slice.submit();


Retry: 9, Time: 205 sec


ID,bf36a948-0886-4275-9d16-62976b3690e9
Name,P4DPDK_lab4_vnic
Lease Expiration (UTC),2024-09-04 14:46:34 +0000
Lease Start (UTC),2024-09-03 14:46:34 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
78fe1a9a-7d08-4cab-bed1-54f95c076678,server1,4,8,100,default_ubuntu_20,qcow2,newy-w2.fabric-testbed.net,NEWY,ubuntu,2001:400:a100:3040:f816:3eff:fec7:715,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3040:f816:3eff:fec7:715,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the DPDK library, the P4 compiler (p4c), and all needed dependencies.

### Step 4.1 Getting the server node
The command below gets the fablib node that server1 is associated with.

In [7]:
server1 = slice.get_node(name="server1")

### Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [8]:
from ipaddress import ip_address, IPv6Address

if type(ip_address(server1.get_management_ip())) is IPv6Address:
    server1.upload_file('scripts/nat64.sh', 'nat64.sh')
    stdout, stderr = server1.execute(f'chmod +x nat64.sh && ./nat64.sh', quiet=True)

### Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [9]:
stdout, stderr = server1.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools', quiet = True)
stdout, stderr = server1.execute(f'sudo pip3 install meson ninja scapy', quiet = True)

### Step 4.4 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers. In this lab, we are building a modified version of DPDK in which we enabled logs in the terminal to have a better understanding of the behavior of the built pipeline. Note that printing out logs on the terminal degrades performance. Therefore, with applications where high performance is needed, logging should be disabled.  

In [10]:
stdout, stderr = server1.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
server1.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
server1.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
stdout, stderr = server1.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

### Step 4.5 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline

In [11]:
stdout, stderr = server1.execute('git clone https://github.com/CILab-USC/p4c.git', quiet = True)
stdout, stderr = server1.execute('sudo apt-get install -y cmake g++ git automake libtool libgc-dev bison flex libfl-dev libboost-dev libboost-iostreams-dev libboost-graph-dev llvm pkg-config python3 python3-pip tcpdump', quiet = True)
stdout, stderr = server1.execute('cd p4c && pip3 install --user -r requirements.txt && mkdir build && cd build && cmake .. && make -j4  && sudo make install', quiet = True)

# Step 5: Writing and compiling the P4 code
This section demonstrates how to define custom headers in a P4 program. It also shows how to use constants and typedefs to make the program more readable.

## Step 5.1: Programming the exact table in the control block

Click on [control.p4](./labs_files/lab4/control.p4) to open the code in the editor.

<img src="./labs_files/lab4/figs/5_01_01.png" width="700px"><br>
We can see that the control.p4 declares a control block named MainControl. Note that the body of the control block is empty. Our objective is to define a P4 table and its actions, and then invoke them inside the block.

<hr>

We will start by defining the possible actions that a table will call. In this simple forwarding program, we have two actions: 

•	```forward```: This action will be used to forward the packet out of a port. 
•	```drop```: This action will be used to drop the packet.

Now we will define the behavior of the ```forward``` action. Insert the code below inside the ```MainControl``` control block. 

    action forward (PortId_t port_id) {
        send_to_port(port_id);
    }

<img src="./labs_files/lab4/figs/5_01_02.png" width="700"><br>

The action ```forward``` accepts as a parameter the port number (```PortId_t port_id```) to be used by the pipeline to forward the packet. The ```send_to_port``` is a function that takes the port number as an input. Therefore, when the forward action is executed, the packet will be sent out of the port number specified as a parameter.

<hr>

Now we will define the drop action. Insert the code below inside the MainControl control block. 
    
    action drop() {
        drop_packet();
    }

<img src="./labs_files/lab4/figs/5_01_03.png" width="700"><br>

The drop() action invokes a primitive action drop_packet() that causes the packet to be dropped.

<hr>
Now we will define the table named forwarding. Write the following piece of code inside the body of the MainControl control block. 

    table forwarding {
    
    }

<img src="./labs_files/lab4/figs/5_01_04.png" width="700"><br>

Tables require keys and actions. In the next step, we will define a key.

<hr>

Add the following code inside the forwarding table. 

    key = {
        hdr.ipv4.dstAddr:exact;
    }

<img src="./labs_files/lab4/figs/5_01_05.png" width="700"><br>

The inserted code specifies that the destination IPv4 address of a packet (```hdr.ipv4.dstAddr```) will be used as a key in the table. Also, the match type is ```exact```, denoting that the value of the destination IP address will be matched as is against a value specified later in the control plane.

<hr>

Add the following code inside the forwarding table to list the possible actions that will be used in this table.

    actions = {
        forward;
        drop;
    }

<img src="./labs_files/lab4/figs/5_01_06.png" width="700"><br>

<hr>

Add the following code inside the forwarding table. The size keyword specifies the maximum number of entries that can be inserted into this table from the control plane.

    size = 1024;

<img src="./labs_files/lab4/figs/5_01_07.png" width="700"><br>

The code above denotes that a maximum of 1024 rules can be inserted into the table.

<hr>
Add the following code inside the MainControl block. The apply block defines the sequential flow of packet processing. It is required in every control block, otherwise the program will not compile. It describes in order, the sequence of tables to be invoked, among other packet processing instructions. 

    apply {
        if(hdr.ipv4.isValid()) {
    	 forwarding.apply();
        }else{
        	 drop(); 
        }
    }

<img src="./labs_files/lab4/figs/5_01_08.png" width="700"><br>

In the code above, we are calling the table forwarding (```forwarding.apply()```) only if the IPv4 header is valid (```if (hdr.ipv4.isValid()```), otherwise the packet is dropped. The validity of the header is set if the parser successfully parsed said header (see parser.p4 for a recap on the parser details).

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.2: Compiling the P4 code
To upload all needed P4 codes and compile main.p4, issue the following command.

In [13]:
server1.upload_file('labs_files/lab4/headers.p4','headers.p4')
server1.upload_file('labs_files/lab4/parser.p4','parser.p4')
server1.upload_file('labs_files/lab4/precontrol.p4','precontrol.p4')
server1.upload_file('labs_files/lab4/control.p4','control.p4')
server1.upload_file('labs_files/lab4/deparser.p4','deparser.p4')
server1.upload_file('labs_files/lab4/main.p4','main.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o lab4.spec')
stdout, stderr = server1.execute(f'ls')

 sudo: unable to resolve host server1: Name or service not known
 control.p4
deparser.p4
dpdk
headers.p4
host_tune.sh
lab4.spec
main.p4
nat64.sh
p4c
parser.p4
precontrol.p4


The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab4.spec``` file which is a specification file needed to run the pipeline.

# Step 6: Running the P4-DPDK pipeline and the lab topology
This section shows the steps required to run the P4-DPDK along with building the lab topology. In this lab, the procedure is automated.

## Step 6.1: Uploading required scripts
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

We will also upload the rules.txt file to load and add rules to the forwarding table. The rules.txt file contains rules for exact matching.

In [18]:
server1.upload_file('labs_files/lab4/lab4.cli','lab4.cli')
server1.upload_file('labs_files/lab4/ethdev.io','ethdev.io')
server1.upload_file('labs_files/lab4/run_pipeline.sh','run_pipeline.sh')
server1.upload_file('labs_files/lab4/set_topology.sh','set_topology.sh')
server1.upload_file('labs_files/lab4/sender.py','sender.py')
server1.upload_file('labs_files/lab4/rules.txt','rules.txt')
stdout, stderr = server1.execute(f'chmod +x run_pipeline.sh')
stdout, stderr = server1.execute(f'chmod +x set_topology.sh')
stdout, stderr = server1.execute(f'chmod +x sender.py')

## Step 6.2: Opening a terminal
Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab4/figs/6_02_terminal.gif" width="1000px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

In [15]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3040:f816:3eff:fec7:715'

## Step 6.3: Running the P4-DPDK pipeline
Build and run the pipeline by typing the following command:

    sudo ./run_pipeline.sh
    
<img src="./labs_files/lab4/figs/6_03.png" width="750px"><br>

The ```run_pipeline.sh``` script is a shell script that automates the process of running the P4-DPDK pipeline. In this lab, part of the pipeline output is hidden to display only the relevant logs.

## Step 6.4: Building the lab topology
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [19]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

 sudo: unable to resolve host server1: Name or service not known
 

The set_topology.sh script is a shell script that automates the process of building the lab topology. Two namespaces are built and configured in this step with a virtual device linked to each as shown in the figure below.

<img src="./labs_files/lab1/figs/6_04_ip.png" width="550px"><br>

## Step 6.5: Inspect the rules.txt file
Click on [rules.txt](./labs_files/lab4/rules.txt) to open the code in the editor.

<img src="./labs_files/lab4/figs/6_05.png" width="700px"><br>

The grey box shows the entries loaded to the match action table. Each entry consists of three components.

•	```match <key>```: The key based on which a match is found or not. In the first entry, the key is 192.168.10.1 (0xC0A80A01 in hexadecimal) which is the IP address of host 1. The match key of the second entry is 192.168.10.2 (0xC0A80A01 in hexadecimal) which is the IP address of host 2. <br>
•	```action <action name>```: The action to be executed if there is a match. If a match is found the action to be executed for both entries is the forward action declared in the control.p4 code which was written earlier in this lab. <br>
•	The last item is the action data. In this case, the action data is the port ID. Therefore, if a match is found, the packet is forwarded through port 0 or port 1.


# Step 7: Testing exact matching

### Step 7.1: Sending an ethernet packet from h1 to h2
To test exact matching, we will send a packet with an ethernet, IPv4, and TCP header from h1 to h2 over the destination IP address 192.168.10.2 by running the provided Python script sender.py.

In [20]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.10.2')

. sudo: unable to resolve host server1: Name or service not known
 Sender: h1
Destination IP: 192.168.10.2

Sent 1 packets.


Running the Python script requires two parameters:

•	```-s```: Sender (h1 or h2) <br>
•	```-d```: Destination IP Address


### Step 7.2: Inspect the pipeline terminal

Observe the DPDK logs at the bottom of the terminal. These logs correspond to the packet processing function executed in the .spec file generated when the P4 code is compiled. 

<img src="./labs_files/lab4/figs/7_02.png" width="700px"><br>

The log highlighted in the first grey box corresponds to the match found in the match-action table with table ID 0 which is the forwarding table. Since there is a match, the forwarding action with action ID 1 is executed. The “tx 1 pkt to port 1” log indicated that the forwarding action is properly executed by sending one packet to port 1 which corresponds to host 2.

### Step 7.3: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab4/figs/7_03.png" width="700px"><br>

Execute the command below to avoid having a pipeline instance running in the background.

In [21]:
stdout, stderr = server1.execute(f'sudo pkill -9 pipeline')

 sudo: unable to resolve host server1: Name or service not known
 

# Step 8: Enabling add_on_miss
This section demonstrates how to define custom headers in a P4 program. It also shows how to use constants and typedefs to make the program more readable.

## Step 8.1: Modifying the P4 code

Click on [headers.p4](./labs_files/lab4/headers.p4) to open the code in the editor.

<img src="./labs_files/lab4/figs/8_01_01.png" width="700px"><br>

We can see that the headers.p4 has a designated space in which we will be declaring an additional constant along with a structure. 

<hr>

Now we will define the EXPIRE_TIME_PROFILE_ID constant. Insert the code below in the designated empty space in line 8.

    const ExpireTimeProfileId_t EXPIRE_TIME_PROFILE_ID = (ExpireTimeProfileId_t) 4;
    
<img src="./labs_files/lab4/figs/8_01_02.png" width="700px"><br>

Each P4 target has a set of expire time profiles where each profile represents the time in seconds it takes for an entry to be removed if it has not been a match to the packets being processed. ```EXPIRE_TIME_PROFILE_ID``` is a constant that represents the expire time profile ID of the entry added. This constant is represented in 8 bits since it is cast to the ```ExpireTimeProfileId_t``` typedef. In this lab, we selected the expiration time with profiled ID 4 which is equivalent to 300 seconds. The set of expire time profiles can be found in the .spec file created after the P4 code is compiled.

<hr>

Now we will define the forward_action_keys structure. Insert the code below in the designated empty space in line 9. 

    struct forward_action_keys {
        PortId_t port_id;
    }

<img src="./labs_files/lab4/figs/8_01_03.png" width="700"><br>

```forward_action_keys``` is a structure that contains the inputs (action data) of the forward action declared in the control.py script. In this lab, the forward action takes only the port_id as an action data. The port is cast to the ```PortId_t``` typedef which represents 32 bits.

<hr>

Save the changes by pressing ```Ctrl+s```.

<hr>

Click on [control.p4](./labs_files/lab4/control.p4) to open the code in the editor.

<img src="./labs_files/lab4/figs/8_01_04.png" width="700"><br>

<hr>

Now we will define the ```forward_miss``` action and its behavior. Insert the code below inside the MainControl control block below the defined actions. 

    action forward_miss () {
        forward_action_keys forward_params;
        forward_params.port_id = (PortId_t) 1;
        add_entry(action_name = "forward", action_params = forward_params,
                              expire_time_profile_id = EXPIRE_TIME_PROFILE_ID);
    }

<img src="./labs_files/lab4/figs/8_01_05.png" width="700"><br>

The action ```forward_miss``` does not accept any input parameters from the control plane. This action is executed when no match is found among the entries in the table. Therefore, an entry is added to the table.

First, we are including the ```forward_action_keys``` structure and labeling it as ```forward_params```. Then, we set the port field to be equal to 1 by casting it to the 8-bit representation typedef PortId_t.

The ```add_entry``` function adds the entry to the table by grabbing the key based on which we are matching in the table from the processed packet. In this lab, we are matching against the IPv4 destination address. Therefore, the key would be the IPv4 destination address of the packet received. The add entry function associated the added key with an action specified as ```action_name```. In this application, all added entries will be associated with the forward action defined earlier in this lab. The forward action takes the port ID as a parameter. Therefore, the input parameter (action data) is passed to the ```add_entry``` function by setting the ```action_params``` equal to the defined structure ```forward_params```. Finally, the ```expire_time_profile_id``` is set equal to the EXPIRE_TIME_PROFILE_ID constant defined earlier.

<hr>

In the forwarding table actions, we will substitute the drop action with the forward_miss action. 

    forward_miss;

<img src="./labs_files/lab4/figs/8_01_06.png" width="700"><br>

<hr>

Add the following code inside the forwarding table. 

    add_on_miss = true;
    default_action = forward_miss;

<img src="./labs_files/lab4/figs/8_01_07.png" width="700"><br>

The ```add_on_miss``` parameter is set to true to enable the feature. We also set the ```forward_miss``` as the default action.

Note that the ```add_on_miss``` feature can only be enabled with a table that applies exact matching and that the default action must call the action that contains the ```add_entry``` function. 

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 8.2: Compiling the P4 code
To upload all needed P4 codes and compile main.p4, issue the following command.

In [22]:
server1.upload_file('labs_files/lab4/headers.p4','headers.p4')
server1.upload_file('labs_files/lab4/control.p4','control.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o lab4.spec')
stdout, stderr = server1.execute(f'ls')

 sudo: unable to resolve host server1: Name or service not known
 control.p4
deparser.p4
dpdk
ethdev.io
headers.p4
host_tune.sh
lab4.cli
lab4.spec
main.p4
nat64.sh
p4c
parser.p4
precontrol.p4
rules.txt
run_pipeline.sh
sender.py
set_topology.sh


The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab4.spec``` file which is a specification file needed to run the pipeline.

# Step 9: Running the P4-DPDK pipeline and the lab topology
This section shows the steps required to run the P4-DPDK along with building the lab topology. In this lab, the procedure is automated.

In [14]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.178'

## Step 9.1: Running the P4-DPDK pipeline
Build and run the pipeline by typing the following command:

    sudo ./run_pipeline.sh
    
<img src="./labs_files/lab3/figs/6_03.png" width="650px"><br>

The ```run_pipeline.sh``` script is a shell script that automates the process of running the P4-DPDK pipeline. In this lab, part of the pipeline output is hidden to display only the relevant logs.

When add_on_miss is enabled, the p4c-dpdk compiler translates the defined table in the P4 code to a learner in the .spec file. Therefore, forwarding is no longer a table. This results in an error when the rules are added to a table named forwarding. 

## Step 9.2: Inspecting the .spec file
Click on [lab4.spec](./labs_files/lab4/lab4.spec) to open the code in the editor.

<img src="./labs_files/lab4/figs/9_04.png" width="750px"><br>

Note that there are 8 different expiration time profiles that we can choose from. In this lab, we selected the expiration time profile with profile ID 1 as declared in the constant ```EXPIRE_TIME_PROFILE_ID``` in the headers.p4 script earlier in this lab. Therefore, the expiration time profile with profile ID 4 corresponds to 300 seconds.

## Step 9.3: Building the lab topology
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [23]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

 sudo: unable to resolve host server1: Name or service not known
 

The set_topology.sh script is a shell script that automates the process of building the lab topology. Two namespaces are built and configured in this step with a virtual device linked to each as shown in the figure below.

<img src="./labs_files/lab1/figs/6_04_ip.png" width="550px"><br>

# Step 10: Testing exact matching with add-on-miss enabled

### Step 10.1: Sending an ethernet packet from h1 to h2
To test exact matching with add-on-miss, we will send a packet with an ethernet, IPv4, and TCP header from h1 to h2 over the destination IP address 192.168.10.2 by running the provided Python script sender.py.

In [25]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.10.2')

. sudo: unable to resolve host server1: Name or service not known
 Sender: h1
Destination IP: 192.168.10.2

Sent 1 packets.


### Step 10.2: Inspect the pipeline terminal

Observe the DPDK logs at the bottom of the terminal. These logs correspond to the packet processing function executed in the .spec file generated when the P4 code is compiled. 

<img src="./labs_files/lab4/figs/10_02.png" width="550px"><br>

The log highlighted in the first grey box indicates that no match was found in the learner table. Therefore, the forwarding_miss action with action ID 1 is executed and an entry is added now to the table.

For a more readable output press enter in the terminal a few times (five times) to provide space for the next logs.

### Step 10.3: Re-sending an IP packet from h1 to h2
Now that an entry is added, we will re-send a packet with an ethernet, IPv4, and TCP header from h1 to h2 over the destination IP address 192.168.10.2 by running the provided Python script sender.py.

In [26]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.10.2')

. sudo: unable to resolve host server1: Name or service not known
 Sender: h1
Destination IP: 192.168.10.2

Sent 1 packets.


### Step 10.4: Inspect the pipeline terminal

<img src="./labs_files/lab4/figs/10_04.png" width="650px"><br>

The log highlighted in the first grey box corresponds to the match found in the learner table with table ID 0. This is due to the added entry. Now that a match has been found, the forward action with action ID 0 is executed. The “tx 1 pkt to port 1” log indicated that the forwarding action is properly executed by sending one packet to port 1 as assigned in the forward_miss action.

### Step 10.5: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab4/figs/10_05.png" width="650px"><br>

Note that the logs in the terminal correspond to the code executed for packet processing. 

# Step 11: Delete the slice

This concludes Lab 4. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab4_vnic")
slice.delete()

# References

1.	The P4 Language Consortium, “P4 Portable NIC Architecture (PNA)”, Version 0.5, 2021. [Online]. Available: https://p4.org/p4-spec/docs/PNA.html
2.	P4lang, “pna”, [Online]. Available: https://github.com/p4lang/pna/tree/main?tab=readme-ov-file
3.	“p4c core.p4”. [Online]. Available: https://github.com/p4lang/p4c/blob/main/p4include/core.p4.
4.	The P4 Language Consortium, “P4 Portable Switch Architecture (PSA)”, 2021. [Online]. Available: https://p4.org/p4-spec/docs/PSA.html#sec-match-kinds.